In [ ]:
%pylab inline 

import numpy as np
import numpy
import pandas as pd
import re
import random as rnd
from sklearn.metrics import accuracy_score
import math
import operator
from tqdm import tqdm_notebook as tqdm
import random
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
# read the arrays
a = [float(x) for x in open('titanic_cpp_x_old.txt').read().split()]
cols = int(a[0])
bdx = array(a[1:]).reshape(-1, cols)
bdx.shape

In [ ]:
a = [float(x) for x in open('titanic_cpp_test_x_old.txt').read().split()]
cols = int(a[0])
bdx_test = array(a[1:]).reshape(-1, cols)
bdx_test.shape

In [ ]:
bdy = array([float(x) for x in open('titanic_cpp_y.txt').read().split()])
bdy.shape

In [ ]:
bdy_test = array([float(x) for x in open('titanic_cpp_test_y.txt').read().split()])
bdy_test.shape

In [ ]:
bdx.shape

In [ ]:
# data organization



In [ ]:
num_insts = 19

In [ ]:
num_rows = 1024
num_features = bdx.shape[1]

In [ ]:
num_threads_x = 16384*4
num_threads_y = num_rows

In [ ]:
prog_len = 40
gprogs = np.zeros((num_threads_x, prog_len*2), dtype=np.float32)

In [ ]:
# 2. The data is a single array of 8 features, many rows, and each thread on the Y axis takes an individual row
gdata = np.zeros((num_rows, num_features), dtype=np.float32)
gdatay = np.zeros((num_rows, 1), dtype=np.float32)
gdatatest = np.zeros((num_rows, num_features), dtype=np.float32)
gdatatesty = np.zeros((num_rows, 1), dtype=np.float32)

In [ ]:
# 3. The results - for each thread, single column of results
gresult = np.zeros((num_rows, 1*num_threads_x), dtype=np.float32)
gscores = np.zeros((1, 1*num_threads_x), dtype=np.float32)
gfitness = np.zeros((1, 1*num_threads_x), dtype=np.float32)
gresulttest = np.zeros((num_rows, 1*num_threads_x), dtype=np.float32)
gscorestest = np.zeros((1, 1*num_threads_x), dtype=np.float32)

In [ ]:
sorted_idx = np.zeros((1,num_threads_x), dtype=np.int32)

In [ ]:
# Allocate the buffers in GPU memory
gprogs_gpu = cuda.mem_alloc(gprogs.nbytes)
gprogsnew_gpu = cuda.mem_alloc(gprogs.nbytes) # to hold next generation
sorted_idx_gpu = cuda.mem_alloc(sorted_idx.nbytes)

gdata_gpu = cuda.mem_alloc(gdata.nbytes)
gdatay_gpu = cuda.mem_alloc(gdatay.nbytes)
gresult_gpu = cuda.mem_alloc(gresult.nbytes)
gscores_gpu = cuda.mem_alloc(gscores.nbytes)
gfitness_gpu = cuda.mem_alloc(gfitness.nbytes)

gdatatest_gpu = cuda.mem_alloc(gdatatest.nbytes)
gdatatesty_gpu = cuda.mem_alloc(gdatatesty.nbytes)
gresulttest_gpu = cuda.mem_alloc(gresulttest.nbytes)
gscorestest_gpu = cuda.mem_alloc(gscorestest.nbytes)

In [ ]:
threads_per_block = 16
blocks_x = num_threads_x // threads_per_block
blocks_y = num_threads_y // threads_per_block

In [ ]:
mutation_rate = 0.1
mutation_prob = 0.5
crossover_prob = 0.5
survival_rate = 0.2

In [ ]:
# The GP kernel
s = f"""
#define PROGLEN {prog_len}
#define NUM_FEATURES {num_features}
#define NUM_THREADS_X {num_threads_x}
"""

sf = f"""
#define MUTATION_RATE {mutation_rate}
#define MUTATION_PROB {mutation_prob}
#define CROSSOVER_PROB {crossover_prob}
#define SURVIVAL_RATE {survival_rate}
"""

mod = SourceModule(s + sf + """

#include <curand_kernel.h>

#define NOP 0
#define PUSHV 1
#define PUSHC 2
#define ADD 3
#define MUL 4
#define DIV 5
#define NEG 6
#define MIN 7
#define MAX 8
#define GREATER 9
#define LESS 10
#define EQUAL 11
#define SIN 12
#define COS 13
#define EXP 14
#define LOG 15
#define SQR 16
#define SQRT 17
#define TANH 18



const int nstates = NUM_THREADS_X;
__device__ curandState_t* states[nstates];

extern "C" {

__global__ void initkernel(int seed)
{
    int tidx = threadIdx.x + blockIdx.x * blockDim.x;

    if (tidx < nstates) 
    {
        curandState_t* s = new curandState_t;
        if (s != 0) 
        {
            curand_init(seed, tidx, 0, s);
        }

        states[tidx] = s;
    }
}

__device__ float randfloat(int tidx)
{
    curandState_t s = *states[tidx];
    float x = curand_uniform(&s);
    *states[tidx] = s;
    return x;
}

__device__ unsigned int randint(int tidx, int aX, int aY)
{
    curandState_t s = *states[tidx];
    unsigned int x = curand(&s);
    *states[tidx] = s;
    
    if (aX == aY)
    {
        return aX;
    }
    if (aX == (aY-1))
    {
        // for two consecutives, pick either with equal probability
        if (randfloat(tidx) < 0.5)
        {
            return aX;
        }
        else
        {
            return aY;
        }
    }
    return aX + (x % (aY - aX + 1));
}


__global__ void init_progs(float *gprogs)
{
    int tidx = (blockIdx.x * blockDim.x + threadIdx.x);
    int prog_idx =  tidx * PROGLEN * 2;
    float* start_idx = (gprogs + prog_idx);
    
    for(int ip=0; ip<(PROGLEN*2); ip += 2)
    {
        // random instruction
        float inst = (float)(randint(tidx, 0, TANH));
        float operand = 0;
        if (inst == PUSHV)
        {
            operand = (float)(randint(tidx, 0, NUM_FEATURES-1));
        }
        if (inst == PUSHC)
        {
            operand = (randfloat(tidx) - 0.5) * 5;
        }
        
        
        *(start_idx + ip) = inst; // instruction
        *(start_idx + ip + 1) = operand; // operand
    }
}


// crossover
__device__ void crossover(int tidx, float* mom, float* dad, float* baby)
{
    unsigned int p1 = randint(tidx, 0, PROGLEN - 1) * 2;
	unsigned int p2 = randint(tidx, 0, PROGLEN - 1) * 2;
    
	while (p1 == p2)
	{
        p1 = randint(tidx, 0, PROGLEN - 1) * 2;
        p2 = randint(tidx, 0, PROGLEN - 1) * 2;
	}

	if (p1 > p2)
	{
		int tmp = p1;
		p1 = p2;
		p2 = tmp;
	}
    
    float x;
    
    for (unsigned int i = 0; i < p1; i += 2)
	{
        x = mom[i];
        baby[i] = x;
        x =  mom[i+1];
        baby[i+1] = x;
	}
    
	for (unsigned int i = p1; i < p2; i += 2)
	{
        x = dad[i];
        baby[i] = x;
        x =  dad[i+1];
        baby[i+1] = x;
	}
    
	for (unsigned int i = p2; i < PROGLEN*2; i += 2)
	{
        x = mom[i];
        baby[i] = x;
        x =  mom[i+1];
        baby[i+1] = x;
	}
}


// mutation
__device__ void mutate(int tidx, float* baby)
{
    for (int i = 0; i < PROGLEN*2; i += 2)
	{
		if (randfloat(tidx) < MUTATION_RATE)
		{
			if ((baby[i] == PUSHV) && (randfloat(tidx) < 0.5))
			{
				baby[i + 1] = (float)(randint(tidx, 0, NUM_FEATURES - 1));
			}
			else if ((baby[i] == PUSHC) && (randfloat(tidx) < 0.5))
			{
				baby[i + 1] = (randfloat(tidx) - 0.5) * 5;
			}
			else
			{
                float inst = (float)(randint(tidx, 0, TANH));
                float operand = 0;
                if (inst == PUSHV)
                {
                    operand = (float)(randint(tidx, 0, NUM_FEATURES-1));
                }
                if (inst == PUSHC)
                {
                    operand = (randfloat(tidx) - 0.5) * 5;
                }
                baby[i] = inst;
                baby[i+1] = operand;
			}
		}
	}
}



// will produce one baby per thread and store it in gprogsnew
__global__ void reproduce(int* sorted_idx, float *gprogs, float *gprogsnew)
{
    int tidx = (blockIdx.x * blockDim.x + threadIdx.x);
    int prog_idx =  tidx * PROGLEN * 2;
    float* baby_start_idx = (gprogsnew + prog_idx);
    
    // elitism
    if (tidx == 0)
    {
        int idx = sorted_idx[0];
        float* mom = gprogs + idx*PROGLEN*2;
        
        for(int i=0; i<PROGLEN*2; i++)
        {
            baby_start_idx[i] = mom[i];
        }
        
        return;
    }
    
    // decide whether to do crossover or not
    if (randfloat(tidx) < CROSSOVER_PROB)
    {
        // will do crossover
        int idx1 = sorted_idx[randint(tidx, 0, (int)((float)(NUM_THREADS_X) * SURVIVAL_RATE))];
        int idx2 = sorted_idx[randint(tidx, 0, (int)((float)(NUM_THREADS_X) * SURVIVAL_RATE))];
        
        float* mom = gprogs + idx1*PROGLEN*2;
        float* dad = gprogs + idx2*PROGLEN*2;
        
        crossover(tidx, mom, dad, baby_start_idx);
        mutate(tidx, baby_start_idx);
    }
    else 
    {
        // will only mutate, but copy the baby first
        int idx = sorted_idx[randint(tidx, 0, (int)((float)(NUM_THREADS_X) * SURVIVAL_RATE))];
        float* mom = gprogs + idx*PROGLEN*2;
        
        for(int i=0; i<PROGLEN*2; i++)
        {
            baby_start_idx[i] = mom[i];
        }
        
        if (randfloat(tidx) < MUTATION_PROB)
        {
            mutate(tidx, baby_start_idx);
        }
    }
}


__global__ void compute(int num_real_rows, float *gprogs, float *gdata, float *gresult)
{
    int prog_idx = (blockIdx.x * blockDim.x + threadIdx.x) * PROGLEN * 2;
    int data_row_idx = (blockIdx.y * blockDim.y + threadIdx.y) * NUM_FEATURES;

    int th_y = blockIdx.y * blockDim.y + threadIdx.y;
    int th_x = blockIdx.x * blockDim.x + threadIdx.x;

    int result_idx = NUM_THREADS_X * th_y + th_x; 

    if (th_y >= num_real_rows)
        return; // don't compute where there's no data

    float stack[PROGLEN+4]; // a little bigger just in case

    int sp = 0;

    // push a 0 to the stack
    stack[sp] = 0;
    sp++;

    float x = 0;
    float y = 0;

    float inst = 0;
    float operand = 0;

    /////////////////////////////////
    // Evaluate program on data row
    float* start_idx = (gprogs + prog_idx);

    for(int ip=0; ip<(PROGLEN*2); ip += 2)
    {
        // fetch instruction and operand
        inst = *(start_idx + ip);
        operand = *(start_idx + ip + 1);

        // execute instruction
        if (inst == NOP)
            continue; 

        if (inst == PUSHV)
        {
            // fetch variable from data
            x = *(gdata + data_row_idx + (int)(operand));

            // push to stack
            stack[sp] = x;
            sp++;
        }

        if (inst == PUSHC)
        {
            // push constant to stack
            stack[sp] = operand;
            sp++;
        }

        // math/logic instructions go below
        if ((inst == ADD) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = x + y;
            sp++;
        }

        if ((inst == MUL) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = x * y;
            sp++;
        }

        if ((inst == DIV) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = x / y;
            sp++;
        }

        if ((inst == NEG) && (sp >= 1))
        {
            /*
            // pop one value from stack
            x = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = -x;
            sp++;*/

            stack[sp] = -stack[sp];
        }

        if ((inst == MIN) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = (x > y)?y:x;
            sp++;
        }

        if ((inst == MAX) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = (x > y)?x:y;
            sp++;
        }

        if ((inst == GREATER) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = (float)(x > y);
            sp++;
        }

        if ((inst == LESS) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = (float)(x < y);
            sp++;
        }

        if ((inst == EQUAL) && (sp >= 2))
        {
            // pop two values from stack
            x = stack[sp];
            sp--;
            y = stack[sp];
            sp--;

            // push result to stack
            stack[sp] = (float)(x == y);
            sp++;
        }

        if ((inst == SIN) && (sp >= 1))
        {
            stack[sp] = sin(stack[sp]);
        }

        if ((inst == COS) && (sp >= 1))
        {
            stack[sp] = cos(stack[sp]);
        }

        if ((inst == EXP) && (sp >= 1))
        {
            stack[sp] = exp(stack[sp]);
        }

        if ((inst == LOG) && (sp >= 1))
        {
            stack[sp] = log(stack[sp]);
        }

        if ((inst == SQR) && (sp >= 1))
        {
            stack[sp] = stack[sp]*stack[sp];
        }

        if ((inst == SQRT) && (sp >= 1))
        {
            stack[sp] = sqrt(stack[sp]);
        }

        if ((inst == TANH) && (sp >= 1))
        {
            stack[sp] = tanh(stack[sp]);
        }
    }

    // store result
    float f = 1/(1+exp(-stack[sp]));
    /*if (f > 0.5) 
    {
        f = 1.0;
    }
    else 
    {
        f = 0.0;
    }*/
    
    /*if (isnan(f) || isinf(f))
    {
        f = 0.5;
    }*/
    
    gresult[result_idx] = f;    
}

}
  """, no_extern_c=True)

In [ ]:
func = mod.get_function("compute")
init_func = mod.get_function("initkernel")
initprog_func = mod.get_function("init_progs")
reproduce_func = mod.get_function("reproduce")

In [ ]:
modsc = SourceModule("""

#define NUM_THREADS_X %d

  __global__ void fitness(int num_real_rows, float *gdatay, float *gresult, float *gscores)
  {
      int th_x = blockIdx.x * blockDim.x + threadIdx.x;
      float sum = 0;
      float eps = 1e-15;
      
      for(int i=0; i<num_real_rows; i++)
      {
          float y1 = gdatay[i];
          float y2 = *(gresult + i*NUM_THREADS_X + th_x);
          if (y2 < eps)
          {
              y2 = eps;
          }
          if (y2 > (1-eps))
          {
              y2 = 1-eps;
          }
          if (y1 == 1)
          {
              sum += -log(y2);
          }
          else
          {
              sum += -log(1 - y2);
          }
      }
      
      sum /= num_real_rows;
      if (isnan(sum) || isinf(sum))
      {
          sum = 999999;
      }
      gscores[th_x] = -sum;
  }
  
  __global__ void scores(int num_real_rows, float *gdatay, float *gresult, float *gscores)
  {
      int th_x = blockIdx.x * blockDim.x + threadIdx.x;
      float sum = 0;
      
      for(int i=0; i<num_real_rows; i++)
      {
          float y1 = gdatay[i];
          float y2 = *(gresult + i*NUM_THREADS_X + th_x);
          if (y2 > 0.5)
          {
              y2 = 1.0;
          }
          else 
          {
              y2 = 0.0;
          }
          
          if (y1 == y2)
          {
              sum ++;
          }
      }
      
      sum /= num_real_rows;
      if (isnan(sum) || isinf(sum))
      {
          sum = 0;
      }
      gscores[th_x] = sum;
  }
  """ % (num_threads_x))

In [ ]:
funcsc = modsc.get_function("scores")
funcfit = modsc.get_function("fitness")

In [ ]:
# call this only once
seed = rnd.randint(0, 10000)
init_func(np.int32(seed), block=(threads_per_block, 1, 1), grid=(blocks_x, 1, 1))

In [ ]:
# init first population
initprog_func(gprogs_gpu, block=(threads_per_block, 1, 1), grid=(blocks_x, 1, 1))

In [ ]:
# store data
gdata[0 : bdx.shape[0], :] = bdx
gdatay[0 : bdx.shape[0], :] = bdy.reshape(-1, 1)

gdatatest[0 : bdx_test.shape[0], :] = bdx_test
gdatatesty[0 : bdx_test.shape[0], :] = bdy_test.reshape(-1, 1)

num_real_rows = np.int32(bdx.shape[0])
num_real_rows_test = np.int32(bdx_test.shape[0])

cuda.memcpy_htod(gdata_gpu, gdata)
cuda.memcpy_htod(gdatay_gpu, gdatay)
cuda.memcpy_htod(gdatatest_gpu, gdatatest)
cuda.memcpy_htod(gdatatesty_gpu, gdatatesty)

In [ ]:
# evolution loop
best_ever = 0
for i in tqdm(range(10000000)):
    # evaluate
    func(num_real_rows, gprogs_gpu, gdata_gpu, gresult_gpu, 
         block=(threads_per_block, threads_per_block, 1), grid=(blocks_x, blocks_y, 1))
    # compute fitness
    #funcfit(num_real_rows, gdatay_gpu, gresult_gpu, gfitness_gpu, 
    #        block=(threads_per_block, 1, 1), grid=(blocks_x, 1, 1))
    # compute scores
    funcsc(num_real_rows, gdatay_gpu, gresult_gpu, gscores_gpu, 
            block=(threads_per_block, 1, 1), grid=(blocks_x, 1, 1))
    # get scores
    cuda.memcpy_dtoh(gscores, gscores_gpu)
    accs = gscores.reshape(-1)
    # get fitness
    #cuda.memcpy_dtoh(gfitness, gfitness_gpu)
    #fits = gfitness.reshape(-1)

    best_idx = np.argmax(accs)
    best = accs[best_idx]
    #print('best', best)
    if best > best_ever:
        best_ever = best
        # eval on test data
        func(num_real_rows_test, gprogs_gpu, gdatatest_gpu, gresulttest_gpu, 
             block=(threads_per_block, threads_per_block, 1), grid=(blocks_x, blocks_y, 1))
        # compute scores
        funcsc(num_real_rows_test, gdatatesty_gpu, gresulttest_gpu, gscorestest_gpu, 
                block=(threads_per_block, 1, 1), grid=(blocks_x, 1, 1))
        # get scores
        cuda.memcpy_dtoh(gscorestest, gscorestest_gpu)
        taccs = gscorestest.reshape(-1)
        #print('new best:', best, 'fitness:', fits[best_idx], 'test:',taccs[best_idx])
        print('new best:', best, 'test:',taccs[best_idx])
        
    # sort fitness, put the sorted_idx in the GPU, and reproduce next generation
    sorted_idx = np.argsort(accs)[::-1].astype(np.int32)
    cuda.memcpy_htod(sorted_idx_gpu, sorted_idx)
    
    reproduce_func(sorted_idx_gpu, gprogs_gpu, gprogsnew_gpu, block=(threads_per_block, 1, 1), grid=(blocks_x, 1, 1))
    
    # replace the old pop
    cuda.memcpy_dtod(gprogs_gpu, gprogsnew_gpu, gprogs.nbytes)